# Aspect-Based Sentiment Analysis (ABSA)¶
This notebook contains the code for the second homework of NLP course 2021 at Sapienza, University of Rome.
Author: Leonardo Emili (1802989)

## Setup the environment

In [1]:
#@title General settings
#@markdown ##### If using GDrive to copy files, make sure to name them correctly.
copy_from_drive = True #@param {type:"boolean"}
if copy_from_drive:
    #!cp /content/gdrive/MyDrive/GoogleNews-vectors-negative300.txt ../../data/
    !cp /content/drive/MyDrive/w2v_weights.pth ../../data/

!nvidia-smi

Wed Sep  1 22:21:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Import dependencies
Setup the environment downloading the required resources, the evaluation tool used for this homework, and configure the logger to have useful plots.

In [1]:
!pip install -q wandb pytorch-lightning transformers axial-positional-embedding pytorch-crf

import sys, os
sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath('../../'))

from dataset import ABSADataset
from hw2.evaluate import evaluate_sentiment
from utils import HParams, log_n_samples, download_nltk_resources, pl_trainer, Vectors

download_nltk_resources()
cached_vectors: Vectors = Vectors.from_cached('../../data/w2v_weights.pth')

import wandb
project = 'nlp_hw2'
wandb.login()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
ERROR: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: leonardoemili (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the comman

True

## Task A - Aspect term identification
In this section, various approaches are shown for aspect terms extraction using ground truth labeled data.

### Baseline model (LSTM)

In [ ]:
from utils import simple_collate_fn
ds = ABSADataset.from_path(collate_fn=simple_collate_fn)

run = wandb.init(reinit=True, project=project, tags=['lstm'])
hparams = HParams(
    ds.ner_vocab,
    input_dim=ds.feature_size,
    hidden_dim=512,
    epochs=30,
    dropout=0.5,
    lr=0.2,
    model_name='lstm'
)
model = NERClassifier(hparams)

trainer = trainer = pl_trainer(max_epochs=hparams.epochs)
trainer.fit(model, ds)
trainer.test()

run.finish()

### BERT-based model

In [ ]:
from models.ner_classifier import NERClassifier
from transformers import BertTokenizer
from utils import collate_fn

bert_name = 'bert-base-uncased'
ds = ABSADataset.from_path(collate_fn=collate_fn, tokenizer=BertTokenizer.from_pretrained(bert_name))

run = wandb.init(reinit=True, project=project, tags=['bert_lstm', bert_name])
hparams = HParams(
    ds.ner_vocab,
    hidden_dim=300,
    epochs=30,
    dropout=0.6,
    lr=0.1,
    model_name='bert_lstm',
    bert_name=bert_name
)
model = NERClassifier(hparams)

trainer = pl_trainer(max_epochs=hparams.epochs)
trainer.fit(model, ds)
trainer.test()

run.finish()

### Pretrained NER classifier (HF)

In [ ]:
from transformers import AutoTokenizer
from utils import collate_fn

bert_name = 'dslim/bert-base-NER'
ds = ABSADataset.from_path(collate_fn=collate_fn, tokenizer=AutoTokenizer.from_pretrained(bert_name))

run = wandb.init(reinit=True, project=project, tags=['bert_lstm'])
hparams = HParams(
    ds.ner_vocab,
    hidden_dim=512,
    epochs=30,
    dropout=0.65,
    lr=0.1,
    model_name='bert_lstm',
    bert_name=bert_name
)
model = NERClassifier(hparams)

trainer = pl_trainer(max_epochs=hparams.epochs)
trainer.fit(model, ds)
trainer.test()

run.finish()

## Task B - Aspect term polarity classification
In this section, polarity classification of extracted aspect terms is performed using a dedicated model for the task.

In [ ]:
from transformers import BertTokenizer
from utils import collate_fn
from models.polarity_classifier import PolarityClassifier

bert_name = 'bert-base-uncased'
ds = ABSADataset.from_path(collate_fn=collate_fn, tokenizer=BertTokenizer.from_pretrained(bert_name))

run = wandb.init(reinit=True, project=project, tags=['bert_lstm', bert_name])
hparams = HParams(
    ds.polarity_vocab,
    hidden_dim=100,
    epochs=30,
    dropout=0.6,
    lr=0.2,
    model_name='bert_lstm',
    bert_name=bert_name
)
model = PolarityClassifier(hparams)

trainer = pl_trainer(max_epochs=hparams.epochs)
trainer.fit(model, ds)
trainer.test()

## Aspect identification pipeline - Task A+B
In this section, we address both aspect terms extraction and polarity classification using a single model.

In [ ]:
from transformers import BertTokenizer
from models.absa_classifier import ABSAClassifier
from utils import collate_fn

bert_name = 'bert-base-uncased'
ds = ABSADataset.from_path(
    merge_dev_sets=True,
    collate_fn=collate_fn,
    tokenizer=BertTokenizer.from_pretrained(bert_name),
    extended_bio=True
)

run = wandb.init(reinit=True, project=project, tags=['bert_lstm', bert_name])
hparams = HParams(
    ds.ner_ext_vocab,
    hidden_dim=300,
    epochs=30,
    dropout=0.6,
    lr=0.1,
    model_name='bert_lstm',
    bert_name=bert_name
)
model = ABSAClassifier(hparams)
model.evaluate_callback = evaluate_sentiment

trainer = pl_trainer(max_epochs=hparams.epochs)
trainer.fit(model, ds)
trainer.test()

run.finish()

## Category identification pipeline - Task C+D
In this section, we address both category terms extraction and polarity classification using a single model.

In [ ]:
from transformers import BertTokenizer
from models.category_classifier import CategoryClassifier
from utils import collate_fn

bert_name = 'bert-base-uncased'
ds = ABSADataset.restaurants_from_path(
    collate_fn=collate_fn,
    tokenizer=BertTokenizer.from_pretrained(bert_name)
)

run = wandb.init(reinit=True, project=project, tags=['bert_lstm', bert_name])
hparams = HParams(
    ds.category_ext_vocab,
    hidden_dim=300,
    epochs=30,
    dropout=0.6,
    lr=0.2,
    model_name='bert_lstm',
    bert_name=bert_name
)
model = CategoryClassifier(hparams, evaluate_callback=evaluate_sentiment)

trainer = pl_trainer(max_epochs=hparams.epochs)
trainer.fit(model, ds)
trainer.test()

run.finish()

## ABSA pipeline - A+B+C+D
In this section, we apply **multitask learning** (Caruana, 1996) to solve both tasks A+B and C+D. More details on the implementation in the report.

In [ ]:
from transformers import BertTokenizer
from models.multistep_classifier import MultistepClassifier
from utils import collate_fn

bert_name = 'bert-base-uncased'
ds = ABSADataset.from_path(
    merge_dev_sets=True,
    collate_fn=collate_fn,
    tokenizer=BertTokenizer.from_pretrained(bert_name),
    extended_bio=True,
    use_class_weights=True,
    cached_vectors=cached_vectors,
    batch_size=256
)

run = wandb.init(reinit=True, project=project, tags=['bert_lstm', bert_name])
ab_hparams = HParams(
    ds.ner_ext_vocab,
    hidden_dim=512,
    epochs=50,
    dropout=0.65,
    lr=0.05,
    bert_name=bert_name,
    sentence_encoder='lstm'
)

cd_hparams = HParams(ds.polarity_vocab)

model = MultistepClassifier(ab_hparams, cd_hparams, ds.category_vocab, pos_vocab=ds.pos_vocab, evaluate_callback=evaluate_sentiment, **ds.configs)
trainer = pl_trainer(max_epochs=ab_hparams.epochs, monitor='trainer/val_aspect_sentiment_f1_macro', mode='max', precision=16)
trainer.fit(model, ds)
trainer.test()

run.finish()

## Inference test
In this section, we can perform testing using models trained at previous steps.

### Task A

In [ ]:
from models.ner_classifier import NERClassifier
from transformers import BertTokenizer
from utils import collate_fn

bert_name = 'bert-base-uncased'
ds = ABSADataset.from_path(collate_fn=collate_fn, tokenizer=BertTokenizer.from_pretrained(bert_name))
hparams = HParams(ds.ner_vocab, input_dim=ds.feature_size, hidden_dim=512, model_name='lstm')
best_model_path = 'checkpoints/epoch=11-step=119.ckpt'
ner_model = NERClassifier.load_from_checkpoint(best_model_path, hparams=hparams)
trainer = pl_trainer()
trainer.test(ner_model, ds.test_dataloader())

### Task B

In [ ]:
from transformers import BertTokenizer
from utils import collate_fn
from models.polarity_classifier import PolarityClassifier

bert_name = 'bert-base-uncased'
ds = ABSADataset.from_path(collate_fn=collate_fn, tokenizer=BertTokenizer.from_pretrained(bert_name))
hparams = HParams(ds.polarity_vocab, hidden_dim=100, model_name='bert_lstm', bert_name=bert_name)
best_model_path = 'checkpoints/polarity_test.ckpt'
polarity_model = PolarityClassifier.load_from_checkpoint(best_model_path, hparams=hparams)
trainer = pl_trainer()
trainer.test(polarity_model, ds.test_dataloader())

### Task A+B

In [ ]:
from transformers import BertTokenizer
from models.absa_classifier import ABSAClassifier
from utils import collate_fn

bert_name = 'bert-base-uncased'
ds = ABSADataset.from_path(
    merge_dev_sets=False,
    collate_fn=collate_fn,
    tokenizer=BertTokenizer.from_pretrained(bert_name),
    extended_bio=True
)

hparams = HParams(
    ds.ner_ext_vocab,
    hidden_dim=300,
    epochs=30,
    dropout=0.6,
    lr=0.1,
    model_name='bert_lstm',
    bert_name=bert_name
)

best_model_path = 'checkpoints/absa_test_3.ckpt'
best_model_path = 'checkpoints/epoch=11-step=239.ckpt'
model = ABSAClassifier.load_from_checkpoint(best_model_path, hparams=hparams)

trainer = pl_trainer(max_epochs=hparams.epochs)
trainer.test(model, ds)

In [ ]:
from models.absa_classifier import AspectMultistepClassifier, model_from
from transformers import BertTokenizer
from utils import collate_fn

bert_name = 'bert-base-uncased'
ds = ABSADataset.from_path(
    collate_fn=collate_fn,
    tokenizer=BertTokenizer.from_pretrained(bert_name),
    extended_bio=True
)

run = wandb.init(reinit=True, project=project, tags=['bert_lstm', bert_name])
ner_hparams = HParams(
    ds.ner_vocab,
    hidden_dim=300,
    epochs=30,
    dropout=0.6,
    lr=0.1,
    model_name='bert_lstm',
    bert_name=bert_name,
    ner_model_path='checkpoints/ner_classifier.ckpt'
)
polarity_hparams = HParams(
    ds.polarity_vocab,
    hidden_dim=100,
    epochs=30,
    dropout=0.6,
    lr=0.2,
    model_name='bert_lstm',
    bert_name=bert_name,
    polarity_model_path='checkpoints/polarity_classifier.ckpt'
)

model = model_from(ner_hparams, polarity_hparams)

trainer = pl_trainer(max_epochs=ner_hparams.epochs)
trainer.test(model, ds.test_dataloader())

run.finish()

### Task C+D

In [ ]:
from transformers import BertTokenizer
from models.category_classifier import CategoryClassifier
from utils import collate_fn

bert_name = 'bert-base-uncased'
ds = ABSADataset.restaurants_from_path(
    collate_fn=collate_fn,
    tokenizer=BertTokenizer.from_pretrained(bert_name)
)

hparams = HParams(
    ds.category_ext_vocab,
    hidden_dim=300,
    epochs=30,
    dropout=0.6,
    lr=0.1,
    model_name='bert_lstm',
    bert_name=bert_name
)

best_model_path = 'checkpoints/epoch=29-step=299.ckpt'
model = CategoryClassifier.load_from_checkpoint(best_model_path, hparams=hparams, evaluate_callback=evaluate_sentiment)
trainer = pl_trainer(max_epochs=hparams.epochs)
trainer.test(model, ds)

### Multistep Testing (A+B+C+D)

In [ ]:
from transformers import BertTokenizer
from models.multistep_classifier import MultistepClassifier
from utils import collate_fn

bert_name = 'bert-base-uncased'
ds = ABSADataset.restaurants_from_path(
    collate_fn=collate_fn,
    tokenizer=BertTokenizer.from_pretrained(bert_name),
    extended_bio=True
)

ab_hparams = HParams(
    ds.ner_ext_vocab,
    hidden_dim=512,
    epochs=30,
    dropout=0.6,
    lr=0.1,
    model_name='bert_lstm',
    bert_name=bert_name
)

cd_hparams = HParams(
    ds.polarity_vocab,
    hidden_dim=512,
    epochs=30,
    dropout=0.6,
    lr=0.1,
    model_name='bert_lstm',
    bert_name=bert_name
)

model = MultistepClassifier.load_from_checkpoint(
    'checkpoints/epoch=15-step=319.ckpt',
    ab_hparams=ab_hparams,
    cd_hparams=cd_hparams,
    category_vocab=ds.category_vocab,
    pos_vocab=ds.pos_vocab,
    evaluate_callback=evaluate_sentiment,
    strict=False
)
trainer = pl_trainer(max_epochs=ab_hparams.epochs)
trainer.test(model, ds)